#  Install Pytorch

In [3]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117


# Install transformers, requests, beautifulsoup, pandas, numpy

In [9]:
!pip install transformers requests beautifulsoup4 pandas numpy


# Import Packages

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# Model and Tokenizer

In [11]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

C:\Users\mahes\anaconda3\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mahes\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

# Sentiment Analysis

In [12]:
tokens = tokenizer.encode('It was good but could be better', return_tensors = "pt")

In [15]:
result = model(tokens)

In [16]:
result.logits

tensor([[-1.8420,  0.1273,  2.0554,  1.1315, -1.2853]],
       grad_fn=<AddmmBackward0>)

Assigning an Integer value to the result

In [17]:
int(torch.argmax(result.logits))+1

3

Input can be changed and an integer value from 1 to 5 will be assigned by calculating sentiment. 1 = low, 5 = high

# Scraping reviews and Analysing sentiment

In [21]:
r= requests.get('https://www.yelp.com/biz/montesacro-soma-san-francisco')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class' : regex})
reviews = [result.text for result in results]

In [22]:
reviews

["Everything, great food, adorable place, excellent wine, and a very hot wait staff (Italian accent anyone?). \xa0We only got appetizers as the pinsa was gluten-aware and not Celiac gluten free. But oh what amazing appetizers they were! Also very nice decor in a neat building with some wonderful leftovers of old SF interspersed amongst the nouveau Soviet architecture that's going up everywhere. Not being able to order the pinsa meant the cost was a little higher, so I couldn't afford it on the reg, but I will *definitely* be returning.",
 "Overall Experience: 4.5 Food: 4Service: 5Atmosphere: 4Value: 4We walked here and the area is a bit sketchy which was a little bit of a downer. The waiter mixed up our food with the table next to us so both my boyfriend and I had the wrong pinsa. I brought up the issue with the waiter and he kindly made me a new one despite me saying that it was okay (this was before we realized my boyfriends order was wrong too). They very kindly gave us the pinsa fo

Import Pandas and numpy to load reviews into Dataframe and Score

In [23]:
import numpy as np
import pandas as pd

In [24]:
df = pd.DataFrame(np.array(reviews), columns = ['review'])

In [25]:
df['review'].iloc[0]b

"Everything, great food, adorable place, excellent wine, and a very hot wait staff (Italian accent anyone?). \xa0We only got appetizers as the pinsa was gluten-aware and not Celiac gluten free. But oh what amazing appetizers they were! Also very nice decor in a neat building with some wonderful leftovers of old SF interspersed amongst the nouveau Soviet architecture that's going up everywhere. Not being able to order the pinsa meant the cost was a little higher, so I couldn't afford it on the reg, but I will *definitely* be returning."

In [26]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors = 'pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [28]:
sentiment_score(df['review'].iloc[1])

4

In [30]:
df['sentiment'] = df['review'].apply(lambda x : sentiment_score(x[:512]))

In [31]:
df

,review,sentiment
0,"Everything, great food, adorable place, excell...",5
1,Overall Experience: 4.5 Food: 4Service: 5Atmos...,4
2,I came here on a Saturday night with a reserva...,4
3,Stopped by this place for some pizza before ca...,3
4,Came here for an after party quick dinner and ...,4
5,Thank you so much for taking the time to revie...,5
6,Really lovely ambiance. The restaurant was dim...,5
7,This cozy and authentic Italian restaurant is ...,5
8,Great wine + pizza! Wow- their pizzas are AHHM...,5
9,Nestled in a charming alleyway off 6th and Mar...,4


In [32]:
df['review'].iloc[5]

'Thank you so much for taking the time to review our restaurant. Please visit us again soon! Grazie! '